# **Dementia Patients -- Analysis and Prediction**
### ***Author : Akhilesh Vyas***
### ****Date : May, 2020****

- <a href='#00'>0. Setup </a>
    - <a href='#00.1'>0.1. Load libraries </a>
    - <a href='#00.2'>0.2. Define paths </a>

- <a href='#01'>1. Data Cleaning and Preprocessing </a>  
    
- <a href='#02'>2. Deep Neural Network Model</a>

- <a href='#03'>3. Result Analysis</a> 

# <a id='00'>0. Setup </a>

## <a id='#00.1'>0.1. Load libraries </a>

In [1]:
import torch
import numpy as np
import pandas as pd

pd.set_option('display.max_columns', None)  
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', -1)

/nfs/home/vyasa/software/pkg/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  import sys


## <a id='#00.2'>0.2. Define paths </a>

In [2]:
%%bash
ls ../../data/

data imputation paper1.pdf
data imputation paper2.pdf
data imputation paper 3.pdf
misdiagnosed.csv
MMSE_Predicted.csv
Optima_Data_Report_Cases_6511_filled.xlsx
patient_com_treat_fea_filled_sel_col.csv
patient_df.csv
patient_df_rnn.csv
patients.csv
X_test.csv
X_train.csv
y_test.csv
y_train.csv


In [3]:
data_path = '../../data/'

# <a id='#01'>1. Data Cleaning and Preprocessing </a>

In [4]:
patient_df = pd.read_csv(data_path+'patient_df_rnn.csv')

print (patient_df.isna().sum()/patient_df.shape[0])
patient_df = patient_df.loc[:, patient_df.isna().sum()/patient_df.shape[0]< 0.4]

patient_df['patient_index'] = patient_df.index
patient_ids = patient_df['patient_id']
patient_eps = patient_df['EPISODE_DATE']

patient_df = patient_df.drop(columns=['EPISODE_DATE', 'MINI_MENTAL_SCORE_PRE'])
patient_df = patient_df.groupby(by='patient_id').transform(lambda x: x.interpolate(method='ffill'))

patient_df['EPISODE_DATE'] = patient_eps
patient_df['patient_id'] = patient_ids

patient_df = patient_df.loc[:, patient_df.isna().sum()/patient_df.shape[0]< 0.2]

patient_df = patient_df.iloc[:, [-1,-2,-3]+ [i for i in range(0, len(patient_df.columns)-3)]]

patient_df = patient_df.fillna(-1)
patient_df.head(5)

MINI_MENTAL_SCORE                                                0.000000
CLINICAL_BACKGROUND:_BODY_MASS_INDEX                             0.348487
ANXIETY/PHOBIC                                                   0.000461
CERBRO-VASCULAR_DISEASE_PRESENT                                  0.000000
DEPRESSIVE_ILLNESS                                               0.000461
DIAGNOSTIC_CODE                                                  0.000461
EST_OF_SEVERITY_OF_DEPRESSION                                    0.000614
EST_SEVERITY_OF_DEMENTIA                                         0.000614
PRIMARY_PSYCHIATRIC_DIAGNOSES                                    0.169866
PETERSEN_MCI                                                     0.000154
PRESENT_STATE_13-46:_(016)_AGE_LEFT_SCHOOL                       0.018277
MOCA:_YEARS_OF_EDUCATION                                         0.885578
PRESENT_STATE_13-46:_(017)_YEARS_IN_FURTHER_EDUCATION            0.021348
HISTORY_PATIENT_74-119:_(076)_HISTORY_

,patient_id,EPISODE_DATE,patient_index,MINI_MENTAL_SCORE,ANXIETY/PHOBIC,CERBRO-VASCULAR_DISEASE_PRESENT,DEPRESSIVE_ILLNESS,DIAGNOSTIC_CODE,EST_OF_SEVERITY_OF_DEPRESSION,EST_SEVERITY_OF_DEMENTIA,PRIMARY_PSYCHIATRIC_DIAGNOSES,PETERSEN_MCI,PRESENT_STATE_13-46:_(016)_AGE_LEFT_SCHOOL,PRESENT_STATE_13-46:_(017)_YEARS_IN_FURTHER_EDUCATION,HISTORY_PATIENT_74-119:_(076)_HISTORY_OF_STROKE,PRESENT_STATE_13-46:_(036)_FEELING_DEPRESSED,INTERVIEWER_OBS_188-212:_(196)_DEPRESSED_MOOD,SEVERITY_OF_DEPRESSION,HISTORY_PATIENT_74-119:_(077)_HISTORY_OF_HEAD_INJURY,PRESENT_STATE_47-73:_(058)_MEMORY_PROBLEM,PRESENT_STATE_47-73:_(062)_DURATION_OF_MEMORY_PROBLEMS,PRESENT_STATE_47-73:_(063)_ONSET_OF_MEMORY_PROBLEMS,PRESENT_STATE_47-73:_(064)_CHANGE_IN_MEMORY_PROBLEMS,CAMCOG_REMOTE_MEMORY_SCORE,CAMCOG_RECENT_MEMORY_SCORE,CAMCOG_LEARNING_MEMORY_SCORE,COGNITIVE_EXAM_120-161:_COGNITIVE_EXAM_120-161,COGNITIVE_EXAM_120-161:_(120)_IDENTIFIES_DAY_OF_WEEK,COGNITIVE_EXAM_120-161:_(121)_IDENTIFIES_DATE,COGNITIVE_EXAM_120-161:_(122)_IDENTIFIES_MONTH,COGNITIVE_EXAM_120-161:_(123)_IDENTIFIES_YEAR,COGNITIVE_EXAM_120-161:_(124)_IDENTIFIES_SEASON,COGNITIVE_EXAM_120-161:_(125)_IDENTIFIES_COUNTY,COGNITIVE_EXAM_120-161:_(126)_IDENTIFIES_TOWN,COGNITIVE_EXAM_120-161:_(127)_IDENTIFIES_STREETS/COUNTRY,COGNITIVE_EXAM_120-161:_(128)_IDENTIFIES_FLOOR,COGNITIVE_EXAM_120-161:_(129)_IDENTIFIES_PRESENT_PLACE,COGNITIVE_EXAM_120-161:_(130)_COMPREHENDS_NOD,COGNITIVE_EXAM_120-161:_(131)_COMPREHENDS_TOUCH,COGNITIVE_EXAM_120-161:_(132)_COMPREHENDS_LOOK,COGNITIVE_EXAM_120-161:_(133)_COMPREHENDS_TAP,COGNITIVE_EXAM_120-161:_(134)_COMPREHENDS_HOTEL,COGNITIVE_EXAM_120-161:_(135)_COMPREHENDS_VILLAGE,COGNITIVE_EXAM_120-161:_(136)_COMPREHENDS_RADIO,COGNITIVE_EXAM_120-161:_(137)_IDENTIFIES_OBJECTS,COGNITIVE_EXAM_120-161:_(137)_IDENTIFIES_OBJECTS:_PENCIL,COGNITIVE_EXAM_120-161:_(137)_IDENTIFIES_OBJECTS:_WATCH,COGNITIVE_EXAM_120-161:_(138)_NAMES_PICTURES,COGNITIVE_EXAM_120-161:_(138)_NAMES_PICTURES:_SHOE,COGNITIVE_EXAM_120-161:_(138)_NAMES_PICTURES:_TYPEWRITER,COGNITIVE_EXAM_120-161:_(138)_NAMES_PICTURES:_SCALES,COGNITIVE_EXAM_120-161:_(138)_NAMES_PICTURES:_SUITCASE,COGNITIVE_EXAM_120-161:_(138)_NAMES_PICTURES:_BAROMETER,COGNITIVE_EXAM_120-161:_(138)_NAMES_PICTURES:_LAMP,COGNITIVE_EXAM_120-161:_(139)_NUMBER_OF_ANIMALS_LISTED,COGNITIVE_EXAM_120-161:_(139)_NUMBER_OF_ANIMALS_LISTED:_SCORE,COGNITIVE_EXAM_120-161:_(140)_DEFINES_HAMMER,COGNITIVE_EXAM_120-161:_(144)_REPETITION,COGNITIVE_EXAM_120-161:_(146)_RECALLS_OBJECTS,COGNITIVE_EXAM_120-161:_(146)_RECALLS_OBJECTS:_SHOE,COGNITIVE_EXAM_120-161:_(146)_RECALLS_OBJECTS:_TYPEWRITER,COGNITIVE_EXAM_120-161:_(146)_RECALLS_OBJECTS:_SCALES,COGNITIVE_EXAM_120-161:_(146)_RECALLS_OBJECTS:_SUITCASE,COGNITIVE_EXAM_120-161:_(146)_RECALLS_OBJECTS:_BAROMETER,COGNITIVE_EXAM_120-161:_(146)_RECALLS_OBJECTS:_LAMP,COGNITIVE_EXAM_120-161:_(147)_RECOGNISES_PICTURES:_SHOE,COGNITIVE_EXAM_120-161:_(147)_RECOGNISES_PICTURES:_SCALES,COGNITIVE_EXAM_120-161:_(147)_RECOGNISES_PICTURES:_BAROMETER,COGNITIVE_EXAM_120-161:_(148)_REMEMBERS_WW1_DATE,COGNITIVE_EXAM_120-161:_(149)_REMEMBERS_WW2_DATE,COGNITIVE_EXAM_120-161:_(150)_REMEMBERS_HITLER,COGNITIVE_EXAM_120-161:_(151)_REMEMBERS_STALIN,COGNITIVE_EXAM_120-161:_(152)_REMEMBERS_MAE_WEST,COGNITIVE_EXAM_120-161:_(153)_REMEMBERS_LINDBERGH,COGNITIVE_EXAM_120-161:_(154)_KNOWS_MONARCH,COGNITIVE_EXAM_120-161:_(155)_KNOWS_HEIR_TO_THRONE,COGNITIVE_EXAM_120-161:_(156)_KNOWS_PRIME_MINISTER,COGNITIVE_EXAM_120-161:_(157)_KNOWS_RECENT_NEWS_ITEM,COGNITIVE_EXAM_120-161:_(158)_REGISTERS_OBJECTS,COGNITIVE_EXAM_120-161:_(158)_REGISTERS_OBJECTS_1:_APPLE,COGNITIVE_EXAM_120-161:_(158)_REGISTERS_OBJECTS_3:_PENNY,COGNITIVE_EXAM_120-161:_(158)_REGISTERS_OBJECTS:_REPEATS,COGNITIVE_EXAM_120-161:_(159)_COUNTING_BACKWARDS,COGNITIVE_EXAM_120-161:_(161)_RECALLS_OBJECTS,COGNITIVE_EXAM_120-161:_(161)_RECALLS_OBJECTS_1:_APPLE,COGNITIVE_EXAM_120-161:_(161)_RECALLS_OBJECTS_2:_TABLE,COGNITIVE_EXAM_120-161:_(161)_RECALLS_OBJECTS_3:_PENNY,COGNITIVE_EXAM_162-187:_COGNITIVE_EXAM_16

In [5]:
# embedding Matrix
embedding_weight = torch.from_numpy(patient_df.iloc[:, 4:].values).float()
print(embedding_weight.shape)
# pairs 
patient_indx_MMS = patient_df.groupby(['patient_id'])['patient_index', 'MINI_MENTAL_SCORE'].agg(lambda x : x.tolist())
patient_indx_MMS['count'] = patient_indx_MMS['patient_index'].apply(lambda x: len(x))


torch.Size([6511, 171])


/nfs/home/vyasa/software/pkg/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """


# <a id='#02'>2. Deep Neural Network Model</a>

In [6]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

import torch
from torch.jit import script, trace
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import csv
import random
import re
import os
import unicodedata
import codecs
from io import open
import itertools
import math
import time
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np

In [7]:
MAX_LENGTH = 10
USE_CUDA = torch.cuda.is_available()
device = torch.device("cuda" if USE_CUDA else "cpu")

In [8]:
pid_list = patient_indx_MMS['patient_index'].values
mmse_list = patient_indx_MMS['MINI_MENTAL_SCORE'].values
pairs = [[pid, mmse] if len(pid) <= 10 else [pid[0:10], mmse[0:10]] for pid, mmse  in zip(pid_list,mmse_list)]
print (pairs[0:10])

def tensorsFromPair(pair):
    input_tensor = torch.tensor(pair[0], dtype=torch.long, device=device).view(-1, 1)
    target_tensor = torch.tensor(pair[1], dtype=torch.float, device=device).view(-1, 1)
    return (input_tensor, target_tensor)

def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=50)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)


In [ ]:
class LSTMModel(nn.Module):
    def __init__(self, hidden_size, output_size, embedding_weight):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding.from_pretrained(embedding_weight)
        self.lstm = nn.LSTM(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        # print(output.dtype, hidden.dtype)
        #output = F.relu(output) # May be not converging due to this 
        # print(output.dtype, hidden.dtype)
        output, hidden = self.lstm(output, hidden)
        #print (output)
        output = self.out(output[0])
        return output, hidden

    def initHidden(self):
        return (torch.zeros(1, 1, self.hidden_size, device=device), torch.zeros(1, 1, self.hidden_size, device=device))

In [ ]:
def train(input_tensor, target_tensor, lstm_model, lstm_model_optimizer, criterion, max_length=MAX_LENGTH):
    
    lstm_model_hidden = lstm_model.initHidden()

    lstm_model_optimizer.zero_grad()

    input_length = input_tensor.size(0)

    loss = 0
    
    for di in range(input_length):
        # print (input_tensor[di].dtype, lstm_model_hidden.dtype)
        lstm_model_output, lstm_model_hidden  = lstm_model(
            input_tensor[di], lstm_model_hidden)
        #print (lstm_model_output.dtype, target_tensor[di].dtype)
        #print (lstm_model_output, target_tensor[di])
        loss += criterion(lstm_model_output[0], target_tensor[di])
        #print ('train:',loss.dtype)

    #print (loss)
    loss.backward()

    lstm_model_optimizer.step()

    return loss.item() / input_length

In [ ]:
def trainIters(lstm_model, n_iters, print_every=1000, plot_every=100, learning_rate=0.001):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    lstm_model_optimizer = optim.Adam(lstm_model.parameters(), lr=learning_rate)
    
    training_pairs = [tensorsFromPair(random.choice(pairs))
                      for i in range(n_iters)]
    
    # need to define loss function
    criterion = nn.MSELoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]
        
        loss = train(input_tensor, target_tensor, lstm_model,
                     lstm_model_optimizer, criterion)
        
        #print ('trainIters:',loss.dtype)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

In [ ]:
hidden_size = 171
output_size = 1

lstm_model = LSTMModel(hidden_size, output_size, embedding_weight).to(device)
#trainIters(lstm_model, 1, print_every=1)
trainIters(lstm_model, 75000, print_every=5000)

# <a id='#03'>3. Result Analysis</a>